In [1]:
import ee
import geemap
import json
import geojson
from IPython.display import Image

import os
import pandas as pd
import requests
from geemap import geojson_to_ee, ee_to_geojson
from ipyleaflet import GeoJSON, Marker, MarkerCluster

In [2]:
Map = geemap.Map(center=[23.834063, 89.398476], zoom=7)
Map

Map(center=[23.834063, 89.398476], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox…

In [3]:
Map.add_basemap("SATELLITE")

In [4]:
poi = ee.Geometry.Point([89.398476, 23.834063])
poi = ee.Feature(poi)

In [5]:
Map.addLayer(poi, {'color':'Red'}, 'point')

In [9]:
area = Map.draw_last_feature

In [10]:
type(area)

ee.feature.Feature

In [11]:
area.getInfo()

{'type': 'Feature',
 'geometry': {'geodesic': False,
  'type': 'Polygon',
  'coordinates': [[[88.341239, 23.038982],
    [88.341239, 25.244385],
    [90.912452, 25.244385],
    [90.912452, 23.038982],
    [88.341239, 23.038982]]]},
 'properties': {}}

In [12]:
geo_area = ee.Geometry(area.getInfo()['geometry'])

In [13]:
geo_area.getInfo()

{'geodesic': False,
 'type': 'Polygon',
 'coordinates': [[[88.341239, 23.038982],
   [88.341239, 25.244385],
   [90.912452, 25.244385],
   [90.912452, 23.038982],
   [88.341239, 23.038982]]]}

In [14]:
type(geo_area)

ee.geometry.Geometry

In [15]:
geo_area.getInfo()['coordinates']

[[[88.341239, 23.038982],
  [88.341239, 25.244385],
  [90.912452, 25.244385],
  [90.912452, 23.038982],
  [88.341239, 23.038982]]]

In [16]:
S2_SR = ee.ImageCollection('COPERNICUS/S2_SR').filterBounds(geo_area).filterDate('2019-01-01', '2020-01-01')

In [17]:
def addNDVI(image):
  ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
  return image.addBands(ndvi)

In [18]:
S2_NDVI = S2_SR.map(addNDVI)

In [19]:
recent_S2 = ee.Image(S2_NDVI.sort('system:time_start', False).first())

In [20]:
NDVIpalette = ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901', '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01', '012E01', '011D01', '011301']

In [21]:
Map.addLayer(recent_S2.select('NDVI'), {'palette': NDVIpalette}, 'Recent Sentinel NDVI')